In [ ]:
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stemmer = WordNetLemmatizer()
def cleaner(document):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', document).strip()
    
#     # remove all single characters
#     document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
#     # Remove single characters from the start
#     document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    # Converting to Lowercase
    document = document.lower()
    re.sub('^his | his ',' donald trump ',document)
    stop = stopwords.words('english')
    stop.remove('who')
    stop.remove('when')
    document = document.split()
    document = [i for i in document if i not in stop]
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)

    return document

In [ ]:
## name finder
def find_name(q):
    q = cleaner(q).split()
    for index,i in enumerate(q):
        if i == 'trump' or i =='madison' or i == 'knauss' or i =='frederick' or i =='kushner' or i =='sophia' or i =='john iii' :
            return q[index-1] + ' ' +q[index]



Logic

In [ ]:
import  kanren.assoccomm as la
from kanren import run,var,fact

## relationship/rules 
relationship = {
"father":
[
    
## Son of Fred
{"fred trump" : "donald trump"},
{"fred trump" : "fred trump jr"},
{"fred trump" : "maryanne trump barry"},
{"fred trump" : "elizabeth trump garu"},
{"fred trump" : "robert trump"},
## Sons of donald trump
{"donald trump": "donald trump jr"},
{"donald trump": "ivanka trump"},
{"donald trump": "barron trump"},
{"donald trump": "eric trump"},
{"donald trump": "tiffany trump"},
{"donald trump jr":"kai madison"},
{"donald trump jr":"chloe sophia"},
{"donald trump jr":"donald john iii"},
{"donald trump jr":"tristan milos"},
{"donald trump jr":"spencer frederick"},
## sons of eric trump, trumps son
{"eric trump":"eric luke trump"},
],
"mother":
[
## Daughter's of Ivanka trump
{"ivanka trump": "arabella rose kushner"},
{"ivanka trump": "joseph frederick kushner"},
{"ivanka trump": "theodore james kushner"},
## Children of Mary Ann trump 
{"mary ann trump": "donald trump"},
{"mary ann trump" : "fred trump jr"},
{"mary ann trump" : "maryanne trump barry"},
{"mary ann trump" : "elizabeth trump garu"},
{"mary ann trump" : "robert trump"},
## Children of donald trump and ivana
{"ivana trump" : "ivanka trump"},
{"ivana trump" : "donald trump jr"},
{"ivana trump" : "eric trump"},
## Children of donald trump and Marla
{"marla maples" : "tiffany trump"},
## Children of donald trump and Melania
{"melania knauss" : "barron trump"},
],
"husband":
[
{"ivana trump" : "donald trump"},
{"marla maples" : "donald trump"},
{"melania knauss" : "donald trump"},
]
}

In [ ]:
## Functions to parse through rules here
from kanren import Relation, facts, run, conde, var, eq
father = Relation()
mother = Relation()
husband = Relation()

# Check if 'x' is the parent of 'y'
def parent(x, y):
    return conde([father(x, y)], [mother(x, y)])

# Check if 'x' is the grandfather of 'y'
def grandparent(x, y):
    temp = var()
    return conde((parent(x, temp), parent(temp, y)))

# Check for sibling relationship between 'a' and 'b'
## temp is parent of x, and temp is parent of y...
## i.e, if donald is parent of eric, and donald is parent of barron, 
def sibling(x, y):
    temp = var()
    return conde((parent(temp, x), parent(temp, y)))

# Check if x is y's uncle
## temp is father of x, and, temp is grandfather of y
## fred is father of robert trump(x), and fred is grandfather of tiffany trump(y)
def uncle(x, y):
    temp = var()
    return conde((parent(temp, x), grandparent(temp, y)))

for item in relationship['father']:
    facts(father, (list(item.keys())[0], list(item.values())[0]))

for item in relationship['mother']:
    facts(mother, (list(item.keys())[0], list(item.values())[0]))

for item in relationship['husband']:
    facts(husband, (list(item.keys())[0], list(item.values())[0]))

x = var()

## QUERY SECTION
In each block, there's a rule for relationship.

In [ ]:
# this block uses rule to find someone's children
## changing that name variable
name = 'Ivanka Trump' ## change 

output = run(0, x, parent(name,x ))
if len(output) == 0 :
    print('I dont understand the question.')

else :
    print("\nList of " + name + "'s children:")
    for item in output:
        print(item)
        ##  the values

In [ ]:
# find someone's grandparents
name = 'Eric Luke Trump' ## change "Ivanka Trump" and example it out
# name_father = run(0, x, father(x, name))[0]

output = run(0, x, grandparent(x,name ))
# output = [x for x in output if x != name_father]
if len(output) == 0:
    print('I dont understand the question.')
else :
    print("\nList of " + name + "'s Grandparents:")
    for item in output:
        print(item)

In [ ]:
# find someone's siblings
name = 'Arabella Rose Kushner' ## changing example "Donald Trump" or "Ivanka Trump" or "Eric Trump"
output = run(0, x, sibling(name,x ))
output = [x for x in output if x != name]
print("\nList of " + name + "'s Siblings:")
for item in output:
    print(item)

In [ ]:
# find someone's uncle or aunt
name = 'Eric Trump'
try : 
    name_father = run(0, x, father(x, name))[0]
    output = run(0, x, uncle(x, name))
    output = [x for x in output if x != name_father]
    print("\nList of " + name + "'s uncles and aunts:")
    for item in output:
        print(item)
except:
    print('I dont understand the question.')

In [ ]:
# find someone's nephew or nephews
name = 'Eric Trump' ## try with "Eric Trump"
try :
    name_sons = run(0, x, father(name, x))[0]
    output = run(0, x, uncle(name, x))
    output = [x for x in output if x not in name_sons]
    print("\nList of " + name + "'s nephew's:")
    for item in output:
        print(item)
except :
    print('I dont understand the question.')


## CORRECT 

In [ ]:
 while True:
    q = raw_input("How can I help you?")
    if q =='stop':
        print('Bye')
        break
    name  = find_name(q)
    q = cleaner(q).split()
    
    if  name == None:
        print('Cannot find names in the quesiton realted to trump. please be more specific.')
        continue
    else :
        print('Name detected, onto the query')
    
    print('the question is parsed as : ',q)
    ## finding parent
    if 'parent' in q or 'father' in q or 'mother' in q:
        output = run(0, x, parent(x,name ))
        if len(output) == 0:
            print('Sorry. I cant find the answer.')
            continue
        else :
            print("\nList of " + name + "'s parent:")
            for item in output:
                print(item)
            continue
    
    ## finding grandparent
    if 'grandparent' in q or 'grandfather' in q or 'grandmother' in q:
        output = run(0, x, grandparent(x,name ))
        if len(output) == 0:
            print('Sorry. I cant find the answer')
            continue
        else :
            print("\nList of " + name + "'s grandparents:")
            for item in output:
                print(item)  
            continue
    
    ## finding grandparent
    if 'grandchild' in q or 'grandchildren' in q or 'grandkids' in q:
        output = run(0, x, grandparent(name,x ))
        if len(output) == 0:
            print('Sorry. I cant find the answer')
            continue
        else :
            print("\nList of " + name + "'s grandparents:")
            for item in output:
                print(item)    
            continue    
                    
    
    ## rule for finding uncle and aunt
    if 'uncle' in q or 'aunt' in q:
        try : 
            name_father = run(0, x, father(x, name))[0]
            output = run(0, x, uncle(x, name))
#             output = [x for x in output if x != name_father]
            print("\nList of " + name + "'s uncles and aunts:")
            for item in output:
                print(item)
            continue
        except:
            print('I dont understand the question.')
            continue
    ## rule for finding nephew
    if 'nephew' in q or 'nephewness' in q :
        try :
            name_sons = run(0, x, father(name, x))[0]
            output = run(0, x, uncle(name, x))
#             output = [x for x in output if x not in name_sons]
            print("\nList of " + name + "'s nephew's:")
            for item in output:
                print(item)
            continue
        except :
            print('I dont understand the question.')
            continue
    ## rule for finding children .. typo sorry
    if 'child' in q or 'children' in q:
        output = run(0, x, parent(name,x ))
        if len(output) == 0 :
            print('I dont understand the question.')
            continue

        else :
            print("\nList of " + name + "'s children:")
            for item in output:
                print(item)
            continue
                ##  the values coming from rules
    ## finding siblings rule
    # find someone's siblings
    if 'sibling' in q or 'brother' in q or 'sister' in q:
         ## change it to say "Donald Trump" or "Ivanka Trump" or "Eric Trump"
        output = run(0, x, sibling(name,x ))
        if len(output) == 0:
            print('I dont understand the question.')
            continue
        else :
#             output = [x for x in output if x != name]
            print("\nList of " + name + "'s Siblings:")
            for item in output:
                print(item)
            continue